<a href="https://colab.research.google.com/github/wozzin/Natural_Language_Processing/blob/main/C_recommend_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install and import the libraries needed
# this script is heavily inspired by Wikidocs
!pip3 install scikit-learn
!pip3 install gdown
from typing import Tuple, List
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import os
import gdown # for downloading the data stored in google drive

In [ ]:
# --- constants --- #
MOVIES_METADATA_CSV_URL = "https://drive.google.com/u/0/uc?id=15I-izNgUJqybrPLK3qlZhLMCuuNcaJhJ&export=download"
MOVIES_METADATA_CSV = "./movies_metadata.csv"  # save the data in the current directory
DOC_SIZE = 2000  # choose the size of the data here (너무 크면 시간이 오래 걸릴 수도 있음!)
TOP_N = 30

In [ ]:
# download the data
gdown.download(url=MOVIES_METADATA_CSV_URL, output=MOVIES_METADATA_CSV, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=15I-izNgUJqybrPLK3qlZhLMCuuNcaJhJ&export=download
To: /content/movies_metadata.csv
34.4MB [00:00, 80.9MB/s]


'./movies_metadata.csv'

In [ ]:
# have a look at the data
movies_df = pd.read_csv(MOVIES_METADATA_CSV, low_memory=False)
movies_df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
# resize the dataframe
movies_df = movies_df.head(DOC_SIZE)
# we don't want any null values. So replace them with an empty string.
# 왜 drop_duplicates?
movies_df['overview'] = movies_df['overview'].fillna('')

In [ ]:
### TODO 1 #### 
# use CountVectorizer & TfidfVectorizer to construct dtm & dtm_tfidf
count = CountVectorizer(binary=True) #tf. binary

dtm: np.ndarray = count.fit_transform(movies_df['overview']).toarray()
dtm_tfidf: np.ndarray = tfidf.fit_transform(movies_df['overview']).toarray()  # from csr_matrix to numpy array.  # 이제는 이거 한줄로 끝내기!
###############
print(dtm)  # should be csr sparse matrix
print(dtm.shape)  # (num_docs, num_terms) 당연히.. 대부분은 0 이겠지! - 다시한번 sparsity를 확인할 수 있다.
print(dtm_tfidf)  # should be csr sparse matrix
print(dtm_tfidf.shape)  # (num_docs, num_terms) 당연히.. 대부분은 0 이겠지! - 다시한번 sparsity를 확인할 수 있다.

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2000, 14195)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2000, 14195)


In [ ]:
# cosine distance -> this is vectorized...
sims_cosine = cosine_similarity(dtm)
sims_cosine_tfidf = cosine_similarity(dtm_tfidf)  # this may take a while... 
print(sims_cosine.shape)  # (num_doc, num_doc)
print(sims_cosine_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)
(2000, 2000)


In [ ]:
# Note that these are distances, now
dists_manhattan_tfidf = manhattan_distances(dtm_tfidf)
print(dists_manhattan_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)


In [ ]:
dists_euclidean_tfidf = euclidean_distances(dtm_tfidf)  # this may take a while... not efficient at all? cosine - it didn't take that much of a time. but manhattan... well that took some time.
print(dists_euclidean_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)


In [ ]:
# this is to retrive the titles
# drop_duplicates: 안하게 되면 같은 영화의 경우,인덱스가 달라질 수 있다.
# 사실 필요는 없을 것.
indices = pd.Series(data=movies_df.index, index=movies_df['title']).drop_duplicates()
indices.head(5)

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [ ]:
batman_idx = indices["Batman"]
batman_returns_idx = indices["Batman Returns"]
toy_story_idx = indices["Toy Story"]
print(sims_cosine_tfidf[batman_idx, batman_returns_idx])
print(sims_cosine_tfidf[batman_idx, toy_story_idx])


0.19155692273625355
0.01586093997388227


In [ ]:
print(type(indices))

<class 'pandas.core.series.Series'>


In [ ]:
indices["Toy Story"] -> 0

In [ ]:
def get_recommendations(title: str,
                        # dists X
                        sims: np.ndarray,
                        indices: pd.Series,
                        data: pd.DataFrame) -> List[Tuple[str, float]]:
    global TOP_N
    ### TODO 2 ###
    # 루프는 사용 X
    # use: np.argsort()[::-1] to recommend titles with scores
    title_idx = indices[title]
    # sims_to_title = sims[title_idx]
    # sims_to_title = sims[title_idx, :] -> 설명을 위해서.
    # shape이 똑같은거 같은데 어떠한 차이가 있는지 궁금합니다~!
    sims_to_title = sims[title_idx, :] # (2000, )
    sims_to_title_sorted: List[float] = np.sort(sims_to_title)[::-1]
    title_indices_sorted = np.argsort(sims_to_title)[::-1]
    titles_sorted: List[str] = data['title'][title_indices_sorted]
    results: List[Tuple[str, float]] = [
        (title, score)
        for title, score in zip(titles_sorted, sims_to_title_sorted)
    ]
    ##############
    return results[:TOP_N]

In [ ]:
my_list = [1, 2, 3]
print(my_list[::-1])

[3, 2, 1]


In [ ]:
# compare the results of dtm with tfidf
# 이유는 다들 알고 계십니다! ㅎㅎ
print("#### dtm + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine, indices, movies_df)):
  print(idx, title, score)
print("#### tfidf + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)

#### dtm + cosine sim ####
0 Batman 0.9999999999999999
1 Cry, the Beloved Country 0.2886751345948129
2 The War at Home 0.2738612787525831
3 Batman & Robin 0.2649064714130089
4 Jane Austen's Mafia! 0.26352313834736496
5 Love & Human Remains 0.2624453295839119
6 Batman Returns 0.24937733402690823
7 Midnight Cowboy 0.2475368857441686
8 The Shadow 0.24618298195866553
9 Highlander 0.24494897427831788
10 Mad Dog Time 0.24056261216234412
11 Apocalypse Now 0.2357022603955159
12 Major Payne 0.23328473740792172
13 Chairman of the Board 0.23145502494313785
14 Jason's Lyric 0.22821773229381925
15 City of Industry 0.22821773229381925
16 Surviving Picasso 0.22742941307367104
17 Oliver & Company 0.22430886163681776
18 The Sting 0.223606797749979
19 A Thousand Acres 0.223606797749979
20 Anna Karenina 0.222717701593687
21 Barcelona 0.22222222222222227
22 Amityville 3-D 0.22019275302527216
23 With Honors 0.21997067253203
24 Frankie Starlight 0.21759706994462233
25 The Apostle 0.2175970699446223
26 A Bro

In [ ]:
### TODO 3 ###
# use different similarity measures and compare the results
# note that dists != sims
print("#### tfidf + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)
print("\n#### tfidf + euclidean dists ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", -dists_euclidean_tfidf, indices, movies_df)):
  print(idx, title, score)
print("\n#### tfidf + manhattan dists ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", -dists_manhattan_tfidf, indices, movies_df)):
  print(idx, title, score) 
##############   

#### tfidf + cosine sim ####
0 Batman 1.0000000000000002
1 Batman Returns 0.19155692273625355
2 Batman & Robin 0.1758295995796183
3 Batman Forever 0.14581071757552017
4 Cry, the Beloved Country 0.11109312245823069
5 B. Monkey 0.10544235266596627
6 The Shadow 0.10514679108660141
7 Sweet Nothing 0.10035634826245518
8 A Month by the Lake 0.10023026852720505
9 Chairman of the Board 0.09920814012584121
10 The War at Home 0.0988387365764903
11 Dune 0.09875401691372578
12 Mediterraneo 0.09803176049698262
13 Return of the Jedi 0.09217825522595026
14 The Pompatus of Love 0.09085969169749389
15 Cyclo 0.08667634812405352
16 Commandments 0.08651635230910235
17 Apocalypse Now 0.08609774357728836
18 Friday the 13th: A New Beginning 0.08605652456139677
19 Dangerous Game 0.08604236366845823
20 Highlander 0.08602706777821761
21 Metropolis 0.08398384733774258
22 Somebody Is Waiting 0.08258337613352316
23 M 0.08193657697542574
24 Mrs. Miniver 0.0811774410090339
25 The Apostle 0.08031383520515435
26 Love 

# 다음의 문제에 답하세요.
> cosine >> euclidean & manhattan 인 이유는 무엇인가요?
1. 고차원 벡터 사이의 유사도를 구하는데, 벡터의 거리를 고려하지 X 각도만을 고려하는 cosine 거리는 차원의저주의 영향을 덜 받는다.
- 서재용09:25
음...컬럼의 수가 너무 많아서?(차원의 저주) 또한 0값이 많기 때문에?
2. 김영학09:25
- 고차원 데이터여서?, **문서의 길이 차이가 많이 나는 경우여서 (2-178)**?결측치의 영향을 안 받아서?(=?벡터의 크기에 영향을 안 받음...각도로만 유사도 측정=?)

> 일반적으로 고차원 벡터인 경우, manhattan이 euclidean보다 성능이 좋은 것으로 알려져 있습니다 (슬라이드). 하지만 이 문제에서는 euclidean > manhattan 입니다. 그 이유는 무엇인가요?

- 논문 - "일반적" -> 정규분포, 푸아송 분포... -> [벡터 성분값은 저런 분포를 따른다] "이상치" -> 이상한 값. [1, 1, 1, ,10000]. 
- [벡터 성분 값] -> tfidf. "이상치"- [1, 1, 1, ,10000].
- 안그래도 차원의 저주로 거리개념이 사라지는데, 유클리드 거리 -> tfidf가 높은/낮은 값 증폭. > ㅇ맨해튼. 

- 서재용09:29 0이 많은 데이터이기때문에 값들 사이 최단 거리(euclidean > manhattan)를 구하는게 더 정확도를 높이지 않났나 싶습니다. -> 일반적인 상황에서는 적합하지 않음. 

- 김영학09:30 정확한 이유는 모르겠음...단지, 문서의 길이에 영향을 받은듯함
-> 결측치를 제외하고 맨해튼의 경우 거리가 작게(유사도가 높은) 나온 영화의 줄거리가 대체로 짧음 / ex - 영화 제목 : Honeymoon, 줄거리 - German Comedy
(manhattan과 euclidean의 둘다 결측치가 유사도가 제일 높은걸로 나왔음 -> manhattan과 euclidean의 비교이므로 결측치 제외하고 비교함)
